In [1]:
# import necessary libraries

import torch
from torch import nn
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# check the version of pytorch
torch.__version__

'2.4.1'

In [3]:
%config InlineBackend.figure_format='retina'

In [4]:
# set the random seed
torch.manual_seed(316)

# 1. Build Network

In [11]:
# Use smae architecture with trained model 

class Network(nn.Module):
    def __init__(self):
        super().__init__()

        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        out = self.relu(self.fc1(x))
        out = self.fc2(out)

        return out

model = Network()
print(model)

Network(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
  (relu): ReLU()
)


# 2. Use trained model's weight

In [12]:
# set the file name that contain the weights
file_name = "model_state_dict.pth"

# load the weight and apply to the Neural Network
model.load_state_dict(torch.load(file_name, map_location = torch.device("cpu")))

# set model to evaluation mode
model.eval()

/var/folders/12/d4mjw2q15bl70mzs4czmwlwm0000gn/T/ipykernel_86985/2706242919.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(file_name, m

Network(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
  (relu): ReLU()
)

# 3. Test the model using Testset

## 3-1. Check GPU state

In [13]:
torch.cuda.is_available()

False

In [14]:
torch.cuda.device_count()

0

In [15]:
if torch.cuda.is_available():
    print(torch.cuda.current_device())

In [16]:
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))

In [17]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

## 3-2. Load Trainset

In [19]:
# define transforms for trainset
transform = transforms.Compose(
    [
        transforms.ToTensor(),
    ]
)

# set batch size as 32
batch_size = 32

# download the testset
testset = datasets.MNIST('.', download=True, train=False, transform=transform)

# loader for the testset
testloader = torch.utils.data.DataLoader(
    testset,
    batch_size=batch_size,
    shuffle=True
)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:04<00:00, 2116748.06it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 112835.41it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:02<00:00, 652926.74it/s] 


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 2309434.93it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



## 3-3. Test the model

In [20]:
# set model to evaluation mode
model.eval()

total = 0  # stores the total number of testset
total_corrects = 0  # stores the total number the model predict correctly

with torch.no_grad():

    for images, labels in testloader:

        images, labels = images.to(device), labels.to(device)

        logits = model.forward(images)

        probabilities = logits.softmax(dim=1)

        top_probs, top_classes = probabilities.topk(k=1, dim=1)
        corrects = (top_classes == labels.view(*top_classes.shape))
        corrects = int(torch.sum(corrects).cpu().numpy())
        total_corrects += corrects

        total += labels.size(0)

In [21]:
# calculate the test accuracy (%)
test_accuracy = total_corrects / total * 100.0
print(test_accuracy)

98.05
